In [4]:
! pip install termcolor

Defaulting to user installation because normal site-packages is not writeable


In [5]:
! find /shared/ -type d -name '*tameriqu*'

find: ‘/shared/’: No such file or directory


In [6]:
! ls -alt /data/.cache/output_main_baseline
! cat /data/.cache/output_main_baseline/config.json

total 501868
drwx------ 10 mtaufeeque 1038      4096 Mar 22 08:12 ..
drwxr-xr-x  3 root       root      4096 Mar 15 20:59 .
-rw-r--r--  1 root       root       225 Mar 15 20:59 metrics.json
-rw-r--r--  1 root       root     15163 Mar 15 20:59 README.md
-rw-r--r--  1 root       root     78759 Mar 15 20:59 trainer_state.json
-rw-r--r--  1 root       root       197 Mar 15 20:59 all_results.json
-rw-r--r--  1 root       root       197 Mar 15 20:59 train_results.json
-rw-r--r--  1 root       root      3643 Mar 15 20:59 training_args.bin
-rw-r--r--  1 root       root   2107625 Mar 15 20:59 tokenizer.json
-rw-r--r--  1 root       root    798156 Mar 15 20:59 vocab.json
-rw-r--r--  1 root       root    456356 Mar 15 20:59 merges.txt
-rw-r--r--  1 root       root        99 Mar 15 20:59 special_tokens_map.json
-rw-r--r--  1 root       root       255 Mar 15 20:59 tokenizer_config.json
-rw-r--r--  1 root       root 510398013 Mar 15 20:59 pytorch_model.bin
-rw-r--r--  1 root       root       119 Mar

In [7]:
ft_model = AutoModelForCausalLM.from_pretrained("/data/.cache/output_main_baseline/")
ft_tokenizer = AutoTokenizer.from_pretrained("/data/.cache/output_main_baseline/")

In [1]:
from codebook_features import models, run_clm, evaluation
import numpy as np
import pickle
from tqdm import tqdm
from transformers import GPTNeoXForCausalLM, AutoTokenizer, pipeline, set_seed, AutoModelForCausalLM
from matplotlib import pyplot as plt
from termcolor import colored
import torch

In [ ]:
! du -hs /shared

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
480G	/shared


In [19]:
device = torch.device("cuda")
model_name_or_path = "EleutherAI/pythia-70m-deduped"
# model_name_or_path = "EleutherAI/pythia-410m-deduped"
# pretrained_path = "/shared/pre_proj_comp_cb_wiki_pythia/"
pretrained_path = "/data/outputs/2023-04-24/13-23-43/output_main"
# pretrained_path = "taufeeque/best-cb-model"
# model_name_or_path = "gpt2"
# pretrained_path = "/shared/comp_cb_8_preproj_attn/"
# pretrained_path = "/data/gpt2_cb/"

# pretrained_path = "/shared/gpt2_on_bc/"
orig_cb_model = models.wrap_codebook(model_or_path=model_name_or_path, pretrained_path=pretrained_path)
hooked_kwargs = dict(center_unembed=True,center_writing_weights=True,fold_ln=True,refactor_factored_attn_matrices=False)
cb_model = models.convert_to_hooked_model(model_name_or_path, orig_cb_model, hooked_kwargs=hooked_kwargs)
# cb_model = cb_model.model
cb_model.model.cfg.device = device
cb_model.to(cb_model.model.cfg.device)


# model = HookedTransformer.from_pretrained(
#     "gpt2-small",
#     center_unembed=True,
#     center_writing_weights=True,
#     fold_ln=True,
#     refactor_factored_attn_matrices=True,
# )

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m-deduped into HookedTransformer


HookedTransformerCodebookModel(
  (model): HookedTransformer(
    (embed): Embed()
    (hook_embed): HookPoint()
    (blocks): ModuleList(
      (0-5): 6 x TransformerBlock(
        (ln1): LayerNormPre(
          (hook_scale): HookPoint()
          (hook_normalized): HookPoint()
        )
        (ln2): LayerNormPre(
          (hook_scale): HookPoint()
          (hook_normalized): HookPoint()
        )
        (attn): Attention(
          (hook_k): HookPoint()
          (hook_q): HookPoint()
          (hook_v): HookPoint()
          (hook_z): HookPoint()
          (hook_attn_scores): HookPoint()
          (hook_pattern): HookPoint()
          (hook_result): HookPoint()
          (hook_rot_k): HookPoint()
          (hook_rot_q): HookPoint()
        )
        (mlp): MLPWrapper(
          (module_layer): MLP(
            (hook_pre): HookPoint()
            (hook_post): HookPoint()
          )
          (codebook_layer): CompositionalCodebookLayer(
            (codebook): ModuleList(
     

In [20]:
model = cb_model
tokenizer = model.tokenizer

In [ ]:
prefixes = ["The largest", "One of the most", "your time is limited", "Once upon a time,"]
# prefixes = ["\n = = The Story =", "\n = = Hello =", "\n = = The Hello2 =", "\n = = This is a very long title =", "\n = = Career ="]
best_cb_generator = pipeline('text-generation', model=orig_cb_model.model, tokenizer=cb_model.tokenizer, device=0)
for prefix in prefixes:
    for i in range(3):
        print(best_cb_generator(prefix, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text'])

In [ ]:
prefixes = ["The largest", "One of the most", "your time is limited", "Once upon a time,"]
# prefixes = ["\n = = The Story =", "\n = = Hello =", "\n = = The Hello2 =", "\n = = This is a very long title =", "\n = = Career ="]
best_cb_generator = pipeline('text-generation', model=ft_model, tokenizer=ft_tokenizer, device=0)
for prefix in prefixes:
    for i in range(3):
        print(best_cb_generator(prefix, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text'])

In [ ]:
! sudo -E /opt/conda/bin/python /data/codebook-features/eval.py \
    --model_name_or_path $model_name_or_path --pretrained_path $pretrained_path \
    --dataset_name bookcorpus --dataset_config_name plain_text

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
04/07/2023 07:12:14 - WARNING - codebook_features.run_clm - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
Dataset wikitext downloaded and prepared to /shared/huggingface/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.
Running tokenizer on dataset:  36%|▎| 647000/1801350 [01:31<02:30, 7663.74 examp[WARNING|tokenization_utils_base.py:3561] 2023-04-07 07:14:16,539 >> Token indices sequence length is longer than the specified maximum sequence length for this model (1059 > 1024). Running this sequence through the model will result in inde

In [ ]:
! ls /shared

2023-03-09_23-21-51  2023-04-07_07-12-14     gpt2_on_bc   output_main_baseline
2023-03-09_23-58-22  2023-04-10_08-11-48     huggingface  outputs
2023-03-10_00-07-02  comp_cb_8_preproj_attn  n.txt
2023-03-31_11-51-32  f.txt		     output_main


In [7]:
# base_dir = "2023-02-17_09-27-39/"
# base_dir = "2023-03-02_21-17-36/"
# base_dir = "2023-03-31_11-51-32/"
# base_dir = "2023-04-07_07-12-14/"
# base_dir = "/shared/2023-04-10_08-11-48/"
# base_dir = "/shared/2023-04-14_13-47-57/"
base_dir = "/homedir/2023-04-25_15-00-13/" # mlp: https://wandb.ai/interpretability/gpt2-codebook/runs/v3d1bt77
cb_at = "mlp"
tokens = np.load(base_dir + "tokens.npy", allow_pickle=True)
# cb_acts_complete = np.load("2023-02-17_09-27-39/cb_acts.npy", allow_pickle=True).item()
with open(base_dir + "cb_acts.pkl","rb") as f:
    cb_acts_complete = pickle.load(f)
metrics = np.load(base_dir + "metrics.npy", allow_pickle=True)
num_codes = 10_000

In [8]:
samples = 10_000
tokens_trunc = tokens[:samples]
cb_acts = {}
for k, v in cb_acts_complete.items():
    cb_acts[k] = v[:samples]


In [6]:
print(tokens.shape)

(10000, 1024)


In [11]:
cb_at = "mlp"

In [12]:
print(metrics)
print(list(cb_acts.keys()))
print(cb_acts[f"layer0_{cb_at}_ccb1"].shape)

{'eval_loss': 3.112891674041748, 'eval_accuracy': 0.4186236559139785, 'eval_runtime': 330.4768, 'eval_samples_per_second': 30.259, 'eval_steps_per_second': 0.475, 'eval_multicode_k': 8, 'eval_dead_code_fraction/layer0': 1.0, 'eval_MSE/layer0': 0.0, 'eval_input_norm/layer0': 0.0, 'eval_output_norm/layer0': 0.0, 'eval_dead_code_fraction/layer1': 1.0, 'eval_MSE/layer1': 0.0, 'eval_input_norm/layer1': 0.0, 'eval_output_norm/layer1': 0.0, 'eval_dead_code_fraction/layer2': 1.0, 'eval_MSE/layer2': 0.0, 'eval_input_norm/layer2': 0.0, 'eval_output_norm/layer2': 0.0, 'eval_dead_code_fraction/layer3': 1.0, 'eval_MSE/layer3': 0.0, 'eval_input_norm/layer3': 0.0, 'eval_output_norm/layer3': 0.0, 'eval_dead_code_fraction/layer4': 1.0, 'eval_MSE/layer4': 0.0, 'eval_input_norm/layer4': 0.0, 'eval_output_norm/layer4': 0.0, 'eval_dead_code_fraction/layer5': 1.0, 'eval_MSE/layer5': 0.0, 'eval_input_norm/layer5': 0.0, 'eval_output_norm/layer5': 0.0}
['layer0_mlp_ccb0', 'layer0_mlp_ccb1', 'layer0_mlp_ccb2', 

In [ ]:
k, v = np.unique(cb_acts["layer0_attn_ccb1"], return_counts=True)
plt.bar(k, v)
plt.yscale('log')
plt.show()

v_sortind = np.argsort(-v)
k, v = k[v_sortind], v[v_sortind]
plt.bar(range(len(v)), v)
plt.yscale('log')
plt.show()

In [13]:

def features_to_tokens(cb_key,n=10):
    codebook_ids = cb_acts[cb_key]
    features_tokens = [[] for _ in range(num_codes)]
    for i in tqdm(range(codebook_ids.shape[0])):
        for j in range(codebook_ids.shape[1]):
            for k in range(codebook_ids.shape[2]):
                past_future_tuple = (tokens[i, max(0, j-n):j+1], tokens[i, j+1:j+n])
                features_tokens[codebook_ids[i,j,k]].append(past_future_tuple)

    return features_tokens

def features_to_tokens_fast(cb_key,n=10):
    codebook_ids = cb_acts[cb_key]
    features_tokens = [[] for _ in range(num_codes)]
    for i in tqdm(range(codebook_ids.shape[0])):
        for j in range(codebook_ids.shape[1]):
            for k in range(codebook_ids.shape[2]):
                features_tokens[codebook_ids[i,j,k]].append((i,j))

    return features_tokens


In [14]:
ft_tkns = features_to_tokens_fast(f"layer0_{cb_at}_ccb0")
ft_tkns2 = features_to_tokens_fast(f"layer2_{cb_at}_ccb0")
ft_tkns5 = features_to_tokens_fast(f"layer5_{cb_at}_ccb0")

100%|██████████| 10000/10000 [00:21<00:00, 471.68it/s]


In [14]:
ft_tkns = features_to_tokens_fast("layer0_attn_ccb0")

100%|██████████| 10000/10000 [04:32<00:00, 36.75it/s] 


In [15]:
ft_tkns2 = features_to_tokens_fast("layer2_attn_ccb0")

100%|██████████| 10000/10000 [06:01<00:00, 27.65it/s]  


In [16]:
ft_tkns5 = features_to_tokens_fast("layer5_attn_ccb0")

100%|██████████| 10000/10000 [09:42<00:00, 17.16it/s]  


In [15]:
def tkn_print(ll, n=3):
    for idx, (i, j) in enumerate(ll):
        print(f"{idx}: example - {i}, token - {j}")
        print(tokenizer.decode(tokens[i,max(0,j-n):j]), end='')
        print(colored(tokenizer.decode(tokens[i,j]), 'red'), end='')
        print(tokenizer.decode(tokens[i,j+1:j+n]))
        if idx > 100:
            break
    print(colored("************************************************************", "green"))
    

In [35]:
import plotly.express as px
num_tkns_for_each_code = [len(tkns) for tkns in ft_tkns if len(tkns) < 100]
px.histogram(num_tkns_for_each_code)

In [34]:
total_tkns = tokens.size
print(total_tkns/(10**6))

10.24


In [16]:
def print_ft_tkns(ft_tkns, n=3, start=0, stop=1000):
    for i in range(start, stop):
#         unique_tokens = set(ft_tkns[i])
#         if 0 < len(unique_tokens) < 100:
#             print(i, len(unique_tokens), len(ft_tkns[i]))
#             tkn_print(unique_tokens)
#             print("**********************************")
        tkns = ft_tkns[i]
        if len(tkns) > 0 and len(tkns) < 1000:
            print(f"code - {i}, mapped to {len(tkns)} tokens")
            tkn_print(tkns, n)

In [21]:
print_ft_tkns(ft_tkns,n=10)

code - 7, mapped to 194 tokens
0: example - 2, token - 911
 its formative stages, the typhoon brought high waves to Guam, resulting in one death.
1: example - 18, token - 332
 and succeeds in making it " leap out of the waves like a modern submarine blowing its ballast tanks
2: example - 72, token - 395
, because data was stored as a sequence of acoustic waves propagated through a mercury column, the device '
3: example - 279, token - 631
 Virginia coastline. In addition to minor erosion, waves caused " slight flooding " at <unk>
4: example - 452, token - 68
head ), the wind increased and with it the waves ; with no high sides, the Monitor took
5: example - 576, token - 860
 air : ridge lift, thermals and lee waves. When conditions are favourable, experienced pilots can
6: example - 702, token - 792
 coast of North America, on rocky shores with constant waves. It is one of the few algae that
7: example - 794, token - 254
 mode, two players team up to combat increasingly difficult waves

In [24]:
print_ft_tkns(ft_tkns2, 10, 3000, 4000)

code - 3045, mapped to 49 tokens
0: example - 193, token - 27
 
 Sb 
 2O 
 3 + 6 HF → 2 SbF 
 3 + 3
1: example - 193, token - 101
 
 3 is prepared by dissolving Sb 
 2S 
 3 in hydrochloric acid : 

2: example - 193, token - 120
 
 Sb 
 2S 
 3 + 6 HCl → 2 SbCl 
 3 + 3
3: example - 193, token - 129
 6 HCl → 2 SbCl 
 3 + 3 H 
 2S 
 The pentahal
4: example - 193, token - 130
 HCl → 2 SbCl 
 3 + 3 H 
 2S 
 The pentahalides
5: example - 193, token - 380
 unstable gas stibine, <unk> 
 3 : 
 Sb3 − + 3 H
6: example - 218, token - 510
, possibly by restoring diminished levels of the antioxidant enzymes superoxide dismutase and catalase caused by experimental
7: example - 218, token - 511
 possibly by restoring diminished levels of the antioxidant enzymes superoxide dismutase and catalase caused by experimental exposure
8: example - 604, token - 748
, the histidine residues <unk> the nickel and attach to the column while the untagged components
9: example - 1211, token - 387
yl hexafluoroplatinat

In [23]:
print_ft_tkns(ft_tkns5,n=10)

code - 47, mapped to 1 tokens
0: example - 1736, token - 88
 feng shui ), the fox's power over evil is such that a mere statue of
************************************************************
code - 180, mapped to 122 tokens
0: example - 508, token - 480
 and published by Capcom. It is a spin @-@ off title in the original Mega Man
1: example - 508, token - 481
 published by Capcom. It is a spin @-@ off title in the original Mega Man series
2: example - 762, token - 399
 and five expansions on gaming consoles. There are spin @-@ offs for Windows and Macint
3: example - 762, token - 400
 five expansions on gaming consoles. There are spin @-@ offs for Windows and Macintosh
4: example - 762, token - 401
 expansions on gaming consoles. There are spin @-@ offs for Windows and Macintosh systems
5: example - 1774, token - 938
 in Japan on November 20, 1995. A spin @-@ off novel written by Endo Ak
6: example - 1774, token - 939
 Japan on November 20, 1995. A spin @-@ off novel written by Endo Ak

In [ ]:
import matplotlib.pyplot as plt

def token_to_dist_of_fts(token, layer_key):
    token_idx = tokenizer.convert_tokens_to_ids(token)
    print(token_idx)
    layer_cb_acts = cb_acts[layer_key]
    token_present = tokens == token_idx
    fts_of_token = layer_cb_acts[token_present]
    print(fts_of_token.shape)
    plt.hist(fts_of_token[:, :3], bins=100, label=["1", "2", "3"])
    plt.legend()
    plt.title(f'Features Queried | Layer = {layer_key}')
    plt.show()

token_to_dist_of_fts('ĠAfrica', 'layer0_attn_ccb0')
token_to_dist_of_fts('ĠAfrica', 'layer1_attn_ccb0')
token_to_dist_of_fts('ĠAfrica', 'layer2_attn_ccb0')
token_to_dist_of_fts('ĠAfrica', 'layer3_attn_ccb0')
token_to_dist_of_fts('ĠAfrica', 'layer4_attn_ccb0')
token_to_dist_of_fts('ĠAfrica', 'layer5_attn_ccb0')

In [2]:
model = models.wrap_codebook(model_or_path="EleutherAI/pythia-70m-deduped", pretrained_path="taufeeque/pythia-attn-cb")

In [137]:
prefixes = ["Hello I am", "What is the", "The largest", "One of the most", "Your time is limited", "Keep your friends close, but", "Once upon a time,"]
# prefixes = [""]

In [139]:
orig_generator = pipeline('text-generation', model=orig_model, tokenizer=tokenizer)
for prefix in prefixes:
    for i in range(3):
        print(orig_generator(prefix, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hello I am a non-member of a team in the USA.

My question is: How do I make a roster in one team?

I am not a member of the Team in the USA. I am in the USA and


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hello I am a newbie with a background in UI.
<xmpp> i'm using a simple text editor but i don't use the web page when it's a text editor
<xmpp> and i've already found


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Keep your friends close, but that's what you make of him."

"How do you know that?"

"I never knew you'd be here," she said. "I never expected you to tell me what you do, and


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Once upon a time, my daughter used to be a very active participant in social distancing.

I had a friend who was in a group together who knew his wife, and he was on the run for a bit for it. She was


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Once upon a time, the same is true of the former, at the time of the loss of the former and the present, of the latter.

The problem is with the loss of the former. For example, the loss of the former
Once upon a time, I just found that the
washers were not so simple as to be able to handle the entire
work, and it was not the time for me to simply do the same. I
had a few different questions about


In [140]:
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model=model.model, tokenizer=tokenizer)
for prefix in prefixes:
    for i in range(3):
        print(generator(prefix, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hello I am a child to be a person of the family. " 
 = = = Reception = = = 
 " The episode of the episode " is a guest @-@ cast @-@ action thriller, which includes guest @-@


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Hello I amai's wife ". The first female of the American writer of the American columnist, and was the authorial, having been inspired by his earlier and @-@ fiction that his " character was " more popular in the United States


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Keep your friends close, but to which their families were to have the house. 
 = = = Other things that are used for the Lord of Lord of the Earl of England and his father of William H. HMS, they had been at the


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Once upon a time, the player is having a career ; " the game is a good and the guy. " 
 The game was released in the United States in the World War I, who gave his first role as the first @-@ game


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Once upon a time,, the fate of this was so many of the world's efforts. 
 = = = Geography of the world = = = = 
 The Great East, the first of the Northern United Nations and the World War (
Once upon a time,, these were not seen in " the episode ", but the episode was originally aired on The MHS and the Fox Show. The film premiered on the Fox Show at the Fox Show. The film was written by the film


In [2]:
best_cb_model = models.wrap_codebook(model_or_path="EleutherAI/pythia-70m-deduped", pretrained_path="taufeeque/pythia-100mc")

Some weights of the model checkpoint at taufeeque/pythia-100mc were not used when initializing GPTNeoXCodebookModel: ['model.gpt_neox.layers.2.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.1.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.5.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.0.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.3.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.4.attention.codebook_layer.codebook.0.counts']
- This IS expected if you are initializing GPTNeoXCodebookModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPTNeoXCodebookModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassifica

In [13]:
from codebook_features import models
import numpy as np
import pickle
from tqdm import tqdm
from transformers import GPTNeoXForCausalLM, AutoTokenizer
from matplotlib import pyplot as plt
from termcolor import colored

tokenizer = AutoTokenizer.from_pretrained(
  "EleutherAI/pythia-70m-deduped",
)
orig_model = GPTNeoXForCausalLM.from_pretrained("EleutherAI/pythia-70m-deduped")
model = models.wrap_codebook(model_or_path="EleutherAI/pythia-70m-deduped", pretrained_path="taufeeque/pythia-100mc")

vis = {}


def hook_fn(m, i, o):
    vis[m] = (i, o)

vis_attn = {}
    
def hook_fn_attn(m, i, o):
    vis_attn[m] = (i, o)

    
for name, layer in model.model.gpt_neox.layers._modules.items():
    print(name)
    layer.attention.codebook_layer.register_forward_hook(hook_fn)
    layer.attention.register_forward_hook(hook_fn_attn)


s = "This tokenizer has been trained to treat spaces like parts of the tokens random tokens added here"
inp = tokenizer(s, return_tensors="pt")
out = model(**inp)


Some weights of the model checkpoint at taufeeque/pythia-100mc were not used when initializing GPTNeoXCodebookModel: ['model.gpt_neox.layers.3.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.5.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.2.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.1.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.4.attention.codebook_layer.codebook.0.counts', 'model.gpt_neox.layers.0.attention.codebook_layer.codebook.0.counts']
- This IS expected if you are initializing GPTNeoXCodebookModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPTNeoXCodebookModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassifica

0
1
2
3
4
5


In [14]:
for i, layers in model.all_codebooks.items():
    for cb in layers:
        print(cb.reconstruction_mse)
        print('cb_norm',cb.codebook[0].avg_norm())
#         print(cb.codebook[0].input.norm())
#         print(cb.codebook[0].output.norm())

44.75636291503906
cb_norm 22.606342315673828
150.0145721435547
cb_norm 22.624954223632812
953.9581298828125
cb_norm 22.625259399414062
567.1958618164062
cb_norm 22.624855041503906
977.8589477539062
cb_norm 22.627485275268555
3250.580322265625
cb_norm 22.615549087524414


In [3]:
model.reset_codebook_metrics()

In [22]:
import importlib
importlib.reload(models)


<module 'codebook_features.models' from '/home/taufeeque/codebook-features/codebook_features/models.py'>

In [ ]:
24.59918189048767
58.6542329788208
1281.2109985351562
1219.846923828125
2715.999969482422
8037.982604980469


In [3]:
model = best_cb_model

In [24]:
print(len(vis))
print(vis.keys())


6
dict_keys([CompositionalCodebookLayer(
  (codebook): ModuleList(
    (0): CodebookLayer(
      (codebook): Embedding(10000, 512)
    )
  )
), CompositionalCodebookLayer(
  (codebook): ModuleList(
    (0): CodebookLayer(
      (codebook): Embedding(10000, 512)
    )
  )
), CompositionalCodebookLayer(
  (codebook): ModuleList(
    (0): CodebookLayer(
      (codebook): Embedding(10000, 512)
    )
  )
), CompositionalCodebookLayer(
  (codebook): ModuleList(
    (0): CodebookLayer(
      (codebook): Embedding(10000, 512)
    )
  )
), CompositionalCodebookLayer(
  (codebook): ModuleList(
    (0): CodebookLayer(
      (codebook): Embedding(10000, 512)
    )
  )
), CompositionalCodebookLayer(
  (codebook): ModuleList(
    (0): CodebookLayer(
      (codebook): Embedding(10000, 512)
    )
  )
)])


In [72]:
for module, (i, o) in vis.items():
    i, o = i[0], o
    print(i.norm(p=2,dim=-1).mean().item(), o.norm(p=2,dim=-1).mean().item())
    print((((i-o)).norm(p=2,dim=-1)**2).mean().item())

7.211728572845459 2.2341723442077637
44.75636291503906
13.70687484741211 2.742445230484009
150.0145721435547
32.99772262573242 3.2736246585845947
953.9581298828125
25.82971954345703 3.1037824153900146
567.1959228515625
33.30326843261719 3.3043413162231445
977.8590698242188
58.66970443725586 3.357837438583374
3250.580322265625


In [11]:
# model.disable_codebooks()
model.reset_codebook_metrics()
vis_attn = {}
out = model(**inp)

for module, (i, o) in vis_attn.items():
    i, o = i[0], o[0]
    print(o.norm(p=2,dim=-1).mean().item())

7.211728572845459
11.842400550842285
29.040924072265625
22.245229721069336
13.874805450439453
21.93659019470215


In [10]:
model.all_codebooks
# wrapper = list(vis_attn.keys())[0]
# print(wrapper.snap)
for layer in vis_attn:
    layer.snap = False

In [44]:
def get_norm(t):
    return t.norm(p=2,dim=-1).mean().item()

In [48]:
last_cb = model.all_codebooks[5][0].codebook[0]

last_o = last_cb(i)
print(get_norm(last_o))

3.357837438583374


In [53]:
import torch
logits = -torch.cdist(i, last_cb.codebook.weight, p=2)
_, codebook_ids = logits.topk(models.BaseSnapFunction.k, dim=-1)
outputs = torch.nn.functional.embedding(codebook_ids, last_cb.codebook.weight)

In [56]:
print(get_norm(outputs))
outputs_avg = outputs.sum(dim=-2) / models.BaseSnapFunction.k
print(get_norm(outputs_avg))

22.306095123291016
3.357837438583374


In [57]:
outputs.shape

torch.Size([1, 18, 100, 512])

In [58]:
outputs_avg.shape

torch.Size([1, 18, 512])

In [71]:
orig_vis = {}


def orig_hook_fn(m, i, o):
    orig_vis[m] = (i, o)

for name, layer in orig_model.gpt_neox.layers._modules.items():
    print(name)
    layer.attention.register_forward_hook(orig_hook_fn)

orig_out = orig_model(**inp)

0
1
2
3
4
5


In [70]:
for module, (i, o) in orig_vis.items():
    i, o = i[0], o[0]
    print(o.norm(p=2,dim=-1).mean().item())

3.2817928791046143
3.281010866165161
4.416781425476074
3.543278455734253
2.703380584716797
14.310317993164062


In [65]:
orig_vis.keys()

dict_keys([])

In [ ]:
import re
pattern = "\n\s*=[^=]*=\s*\n"
def color_pattern(s, p=pattern):
    """colors p in s using termcolor"""
    return re.sub(p, colored(r"\g<0>", "blue", attrs=['bold']), s)

# for idx in range(len(tokens)):
#     example = tokens[idx]
#     print(color_pattern(tokenizer.decode(example)))
#     if idx > 100:
#         break

for idx in range(len(tokens)):
    example = tokens[idx]
    s = tokenizer.decode(example)
    matches = re.findall(pattern, s)
    if len(matches) > 0:
        print(idx, matches)

In [23]:
generator = pipeline('text-generation', model=model.model, tokenizer=tokenizer)
prefix = "When Mary and John went to the store, John gave a drink to"
print(generator(prefix, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


When Mary and John went to the store, John gave a drink to a family member. 
 = = = The householder = = = 
 The house was built by the architect William Horton, in the early 17th century. When the house


In [19]:
orig_generator = pipeline('text-generation', model=orig_model, tokenizer=tokenizer)
print(orig_generator(prefix, max_length=50, do_sample=True, temperature=0.7)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


When Mary and John went to the store, John gave a drink to Mary’s mother of the children and said, “We’re going to get you something nice for me and I’m going to get you something good for me and I
